In [51]:
import pandas as pd
import numpy as np


In [52]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def setup_dataset(text):
  import shutil
  import os
  %cd /content/gdrive/My Drive/Kaggle/
  strings = text.split(' ')[-1]
  folder = strings.split('/')[1]
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/"+folder
  print(strings)
  print(folder)
  !mkdir $folder
  shutil.copy2("kaggle.json","./"+folder+"/kaggle.json")
  %cd $folder
  !kaggle datasets download -d $strings

In [ ]:
setup_dataset("kaggle datasets download -d luizfmatos/reddit-english-depression-related-submissions")

/content/gdrive/My Drive/Kaggle
luizfmatos/reddit-english-depression-related-submissions
reddit-english-depression-related-submissions
mkdir: cannot create directory ‘reddit-english-depression-related-submissions’: File exists
/content/gdrive/My Drive/Kaggle/reddit-english-depression-related-submissions
 51% 9.00M/17.7M [00:00<00:00, 90.5MB/s]
100% 17.7M/17.7M [00:00<00:00, 113MB/s] 


In [ ]:
%ls

kaggle.json  reddit-english-depression-related-submissions.zip


In [ ]:
!unzip reddit-english-depression-related-submissions.zip

Archive:  reddit-english-depression-related-submissions.zip
  inflating: _reddit-posts-gatherer-en.submissions_subset.json  


In [ ]:
df = pd.read_json("_reddit-posts-gatherer-en.submissions_subset.json")

In [ ]:
real_df = df.body

In [ ]:
real_df

0        It came in the mail a few weeks ago, so I put ...
1        It's one of those nights where I want to kill ...
2        I have been suffering with depression for 3 ye...
3        How do you all deal with feelings of isolation...
4        There are people out there enjoying life, are ...
                               ...                        
32160    Fuck throwaways.\n\nThis is a story I've only ...
32161    Even when I should be happy Im not. I feel so ...
32162    January 8th, 2014. The time is approximately 1...
32163    Spending the New Year alone (of course), I dec...
32164    http://www.mayoclinic.org/diseases-conditions/...
Name: body, Length: 32165, dtype: object

In [ ]:
df = real_df

In [ ]:
df.shape

(32165,)

In [ ]:
len(df)

32165

### Convert Emojis to Text

In [ ]:
!pip install emot

In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 6.6MB/s 


In [ ]:
import emoji

In [ ]:
for i in range(len(df)):
  text = df[i]
  new_text = emoji.demojize(text)
  df[i] = new_text

In [55]:
df[100]

'I reach out to try and talk to someone, I get a reply 6 hours later and it\'s just "lol". Sometimes I don\'t get replies at all, then I see them posting all day on social media or whatever...\n\nIs it just me? Am I so unbearably impossible to talk to that my friends can\'t be bothered trying to hold a conversation with me? Am I that annoying and disgusting?\n\nI\'m so tired of feeling this lonely ache in my chest because no one wants to talk to me'

###Converting words to vectors

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [58]:
cv = CountVectorizer(max_df=0.95,stop_words="english",min_df=2)

In [59]:
df_fitt = cv.fit_transform(df)

###Latent Dirichlet Allocation (LDA)

In [63]:
from sklearn.decomposition import LatentDirichletAllocation

In [64]:
lda = LatentDirichletAllocation(n_components=10,random_state=42)

In [65]:
lda.fit(df_fitt)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

###Exploring Topics Assigned by lda

In [67]:
len(cv.get_feature_names())

25700

*Count Vectorizer gave us approx 25700 important words from the given dataset* 

In [71]:
len(lda.components_)

10

LDA Has assigned the 10 topics accordingly as requested

In [73]:
print(lda.components_.shape)

(10, 25700)


In [85]:
topic0 = lda.components_[0]
print(topic0)

[0.42080936 3.71604014 0.10000505 ... 0.10006833 0.1        0.10000105]


In [76]:
topic0.argsort()

array([21718,  8011,  1874, ..., 13410,  6381, 16630])

*Each topic has been represented as probabilities of the features generated by count-vectorizer*

In [84]:
topic0.argsort()[-10:]

array([22933, 24542, 13730, 10975, 13449, 22931, 25433, 13410,  6381,
       16630])



```
# The above word ids are the top 10 words that represent that topic
```



In [87]:
for item in topic0.argsort()[-10:]:
  print(cv.get_feature_names()[item])

think
ve
love
help
like
things
world
life
depression
people


In [90]:
for index,topic in enumerate(lda.components_):
  print("Lets understand topic-{}".format(index))
  print("Top 5 words which represent this topic are :")
  print([cv.get_feature_names()[item] for item in topic.argsort()[-5:]])
  print("---------------------------------------------------------------")

Lets understand topic-0
Top 5 words which represent this topic are :
['things', 'world', 'life', 'depression', 'people']
---------------------------------------------------------------
Lets understand topic-1
Top 5 words which represent this topic are :
['time', 'life', 'work', 've', 'job']
---------------------------------------------------------------
Lets understand topic-2
Top 5 words which represent this topic are :
['doctor', 'medication', 'taking', 've', 'depression']
---------------------------------------------------------------
Lets understand topic-3
Top 5 words which represent this topic are :
['want', 'like', 'life', 'just', 'feel']
---------------------------------------------------------------
Lets understand topic-4
Top 5 words which represent this topic are :
['year', 'class', 'college', 'just', 'school']
---------------------------------------------------------------
Lets understand topic-5
Top 5 words which represent this topic are :
['like', 'bed', 'sleep', 'just', 

###Attaching Labels to discovered topics

In [91]:
topics = lda.transform(df_fitt)

In [92]:
topics.shape

(32165, 10)

Each sentence has been assigned the probability score of being assosiated with some topic out of the 10 that we found out

In [101]:
topics.argmax(axis = 1)

array([1, 3, 8, ..., 9, 2, 3])

In [102]:
topics = topics.argmax(axis = 1)

In [103]:
d = {'sentences':df,'topics':topics}

In [104]:
df = pd.DataFrame(d)

In [105]:
df.head()

,sentences,topics
0,"It came in the mail a few weeks ago, so I put ...",1
1,It's one of those nights where I want to kill ...,3
2,I have been suffering with depression for 3 ye...,8
3,How do you all deal with feelings of isolation...,6
4,"There are people out there enjoying life, are ...",1
